In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

import re
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type,

In [3]:
data = pd.read_csv("rnn_dataset.csv")

### Data Visualization

In [4]:
data.shape

(7889, 2)

In [5]:
data.Title.unique()

array(['App Developer', 'Cloud and DevOps Engineer',
       'Data Scientist and Analyst', 'Full Stack Developer',
       'Machine Learning Engineer', 'Software Developer', 'Web Developer'],
      dtype=object)

In [6]:
total_attributes = len(data.Title.unique())
total_attributes

7

In [7]:
temp1 = data["Title"].unique().tolist()
temp1.sort()
labels = temp1
labels

['App Developer',
 'Cloud and DevOps Engineer',
 'Data Scientist and Analyst',
 'Full Stack Developer',
 'Machine Learning Engineer',
 'Software Developer',
 'Web Developer']

In [8]:
data.Title.value_counts()

Web Developer                 1837
Software Developer            1824
App Developer                 1193
Full Stack Developer          1064
Cloud and DevOps Engineer      950
Data Scientist and Analyst     735
Machine Learning Engineer      286
Name: Title, dtype: int64

In [9]:
## Checking the distribution of data

data['Title'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of People', title='Number of people in each job role')

In [10]:
## This function is not important because it is only for checking whethwe the data description is cleaned or not

def print_plot(index):
    example = data[data.index == index][['Description', 'Title']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Title:', example[1])
print_plot(200)

android developer responsible maintenance application aimed vast number diverse android device primary focus android application integration back service working along side engineer developer working different layer infrastructure therefore commitment collaborative problem solving sophisticated design creating quality product essential  translate design wireframes high quality code design build maintain high performance reusable reliable java code best possible performance quality responsiveness application identify correct bottleneck fix bug strong knowledge android sdk different version android deal different screen size least 2 year kotlin framework strong java intricate concept java familiarity rest api connect android application back service strong knowledge android ui design principle pattern best practice offline storage threading performance tuning design application around natural user interface touch familiarity use additional sensor gyroscope accelerometer knowledge open so

### Further cleaning of Data
- Removing stop words 
- Removing Bad Symbols and spaces

In [11]:
data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#   text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

data['Description'] = data['Description'].apply(clean_text)

## MAKING THE MODEL

In [13]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 200
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['Description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 21862 unique tokens.


In [14]:
X = tokenizer.texts_to_sequences(data['Description'].values)
X = sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (7889, 200)


In [15]:
Y = pd.get_dummies(data['Title']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (7889, 7)


In [16]:
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
#print(X_train.shape,Y_train.shape)
#print(X_test.shape,Y_test.shape)

In [17]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences = True))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(total_attributes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Train on 7100 samples, validate on 789 samples
Epoch 1/10
7100/7100 [==============================] - 76s 11ms/step - loss: 1.5646 - accuracy: 0.3813 - val_loss: 1.7441 - val_accuracy: 0.2548
Epoch 2/10
7100/7100 [==============================] - 81s 11ms/step - loss: 1.1828 - accuracy: 0.5700 - val_loss: 1.3763 - val_accuracy: 0.4981
Epoch 3/10
7100/7100 [==============================] - 87s 12ms/step - loss: 1.0227 - accuracy: 0.6252 - val_loss: 1.6942 - val_accuracy: 0.4588
Epoch 4/10
7100/7100 [==============================] - 88s 12ms/step - loss: 0.9537 - accuracy: 0.6687 - val_loss: 1.6119 - val_accuracy: 0.5158
Epoch 5/10
7100/7100 [==============================] - 86s 12ms/step - loss: 0.8459 - accuracy: 0.7162 - val_loss: 1.3575 - val_accuracy: 0.5589
Epoch 6/10
7100/7100 [==============================] - 90s 13ms/step - loss: 0.7745 - accuracy: 0.7385 - val_loss: 1.5661 - val_accuracy: 0.5488
Epoch 7/10
7100/7100 [==============================] - 91s 13ms/step - loss

### Checking Model Accuracy

In [18]:
accr = model.evaluate(X,Y)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

7889/7889 [==============================] - 25s 3ms/step
Test set
  Loss: 0.695
  Accuracy: 0.779


### Saving weights to .json and .h5 file for loading in Flask

In [19]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")

## Predictions

- Loading .json file and .h5 file for model prediction
- Single Value Prediction

In [20]:
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model1.h5")

In [21]:
labels = ['App Developer',
 'Cloud and DevOps Engineer',
 'Data Scientist and Analyst',
 'Full Stack Developer',
 'Machine Learning Engineer',
 'Software Developer',
 'Web Developer']

In [23]:
def predictions(pred):
    predictions_list = []
    pred_reshape = np.reshape(pred, (total_attributes,1))
    list_pred = pred_reshape.tolist()
    [list_pred[i].append(i+1) for i in range(0,len(list_pred))]
    list_pred.sort(key = lambda x: x[0], reverse = True )
    #print(list_pred)
    for i in range(0,len(list_pred)):
        predictions_list.append(labels[list_pred[i][1] - 1] + " - " + str("{:.2f}".format(list_pred[i][0]*100) + " %"))
    return predictions_list

In [25]:
## new_role is where the summary from the user will come (INPUT)
## last print statement is what is to be printed (OUTPUT)

new_role = ['AI, Machine Learning, Deep Learning, Flask']

seq = tokenizer.texts_to_sequences(new_role)
padded = sequence.pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)

final_predictions = predictions(pred)
print(pd.Series(final_predictions).head(3).tolist())

['Data Scientist and Analyst - 50.66 %', 'Machine Learning Engineer - 22.68 %', 'Full Stack Developer - 15.99 %']
